In [8]:
import os
os.environ["OPENAI_API_KEY"] =

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Dict
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import JsonOutputParser

system_message = """
Given your profound knowledge about words, lexical items and concepts, I'll give you a list of templates of sentences that represents a relation between two words ans a list of words. 
In each template sentence in the list the expression %w1 represents a given word and the expression %w2 represents a related word.
For each word in the given list, you must process all the sentence templates, replacing the %w2 expression with a related word.
Note, please, that not all the given words will use all sentences. Give me just those that make sense from a human point of view.
Don't use metaphorical or mythological relations or interpretations. Use your linguistic (syntactic and grammatical) and conceptual knowledge to complete the sentence.
Here is the list of templates, separated by commas: {relations}.
If possible, structure your response in a JSON-like format. In the response, please make just the word for expression %w2 explicit, not the whole template text.
Try to get from 3 to 5 words for each expression %w2 in a template.
The words are written in Brazilian Portuguese, so the related words must be also writen in Brazilian Portuguese.
\n{format_instructions}\n{query}\n
"""

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

parser = JsonOutputParser()
relations = [
    '%w1 causes %w2',
    '%w1 uses %w2',
    'the occurrence of %w1 can cause emotion %w2',
    '%w2 is caused by %w1 by natural means',
    'the occurrence of %w1 can cause the reaction %w2',
    '%w2 is a typical syntactic subject of %w1',
    '%w2 is a typical syntactic object of %w1',
    '%w2 is a typical semantic agent of %w1',
    '%w2 is a typical semantic patient of %w1',
    '%w1 typically occurs in the context of %w2',
    '%w2 typically occurs in the context of %w1',
    '%w2 is a instrument typically used in the process of %w1',
]

generator_expr = (str(element) for element in relations)
separator = ','
relations_string = separator.join(generator_expr)

prompt = PromptTemplate(
    template=system_message,
    input_variables=["query", "relations"],
    partial_variables={"format_instructions": parser.get_format_instructions(), "relations": relations_string},
)

chain = prompt | llm | parser

response = chain.invoke({"query": "quebrar"})

print(response)

{'quebrar': {'%w1 causes %w2': ['danos', 'fragmentos', 'desgaste'], '%w1 uses %w2': ['força', 'impacto', 'ferramenta'], 'the occurrence of %w1 can cause emotion %w2': ['frustração', 'raiva', 'tristeza'], '%w2 is caused by %w1 by natural means': ['fragmentação', 'danos', 'desgaste'], 'the occurrence of %w1 can cause the reaction %w2': ['reparo', 'substituição', 'limpeza'], '%w2 is a typical syntactic object of %w1': ['vidro', 'prato', 'osso'], '%w2 is a typical semantic agent of %w1': ['martelo', 'queda', 'impacto'], '%w2 is a typical semantic patient of %w1': ['vidro', 'prato', 'osso'], '%w1 typically occurs in the context of %w2': ['acidente', 'construção', 'manutenção'], '%w2 typically occurs in the context of %w1': ['reparo', 'substituição', 'limpeza'], '%w2 is a instrument typically used in the process of %w1': ['martelo', 'pé de cabra', 'alicate']}}
